In [155]:
from mxnet import autograd,nd
import pandas as pd
import numpy as np
data = pd.read_csv("./train.csv")
arr = np.array(data)
arr[:4999,3:6]

array([[281.0, 281.0, 281.0],
       [283.2, 283.2, 283.2],
       [281.4, 281.4, 281.4],
       ...,
       [1538.5, 1553.1, 1532.6],
       [1553.0, 1561.2, 1546.0],
       [1557.4, 1557.7, 1541.3]], dtype=object)

In [156]:
features = arr[:4999,3:]
features = nd.array(features)
features


[[ 281.   281.   281.   282.7    4. ]
 [ 283.2  283.2  283.2  281.1   16. ]
 [ 281.4  281.4  281.4  281.4    0. ]
 ...
 [1538.5 1553.1 1532.6 1561.   238. ]
 [1553.  1561.2 1546.  1563.7  291. ]
 [1557.4 1557.7 1541.3 1556.2  163. ]]
<NDArray 4999x5 @cpu(0)>

In [157]:
labels = arr[1:,6:7]
labels = nd.array(labels)
labels = labels.reshape(shape=[4999])
labels


[ 281.1  281.4  281.9 ... 1563.7 1556.2 1544. ]
<NDArray 4999 @cpu(0)>

In [158]:
from mxnet.gluon import data as gdata

# 超参数
batch_size = 10
dataset = gdata.ArrayDataset(features,labels)
data_iter = gdata.DataLoader(dataset,batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break


[[1.3834e+03 1.3840e+03 1.3642e+03 1.3734e+03 1.7500e+02]
 [1.6470e+03 1.6583e+03 1.6257e+03 1.6481e+03 1.9900e+02]
 [1.3518e+03 1.3518e+03 1.3518e+03 1.3510e+03 1.0000e+00]
 [5.5610e+02 5.5610e+02 5.5610e+02 5.5610e+02 0.0000e+00]
 [2.6400e+02 2.6400e+02 2.6400e+02 2.6400e+02 0.0000e+00]
 [1.2961e+03 1.2961e+03 1.2961e+03 1.2961e+03 2.9000e+01]
 [8.9150e+02 8.9150e+02 8.8450e+02 8.7770e+02 9.2000e+01]
 [1.2793e+03 1.2793e+03 1.2793e+03 1.2793e+03 6.7000e+01]
 [3.9930e+02 3.9930e+02 3.9930e+02 3.9930e+02 0.0000e+00]
 [2.7390e+02 2.7390e+02 2.7390e+02 2.7390e+02 0.0000e+00]]
<NDArray 10x5 @cpu(0)> 
[1371.4 1645.5 1355.2  553.4  263.9 1265.6  882.9 1289.1  393.5  278.4]
<NDArray 10 @cpu(0)>


In [159]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

In [160]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

In [161]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

In [162]:
from mxnet import gluon
#第一个超参数，学习率，并不是越小越好
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.0000000001})

In [163]:
test_data = np.array(pd.read_csv("./test.csv"))
test = nd.array(test_data[0:690,3:])
true_y = test_data[1:691,6:7]
true_y = nd.array(true_y)

In [164]:
def test_acc():
    return loss(net(test),true_y).mean().asnumpy()

In [165]:
num_epochs = 20
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features),labels)
    print('epoch %d,loss: %f test_lost: %f' % (epoch,l.mean().asnumpy(),test_acc()))

epoch 1,loss: 6779.737305 test_lost: 25707.001953
epoch 2,loss: 331.221252 test_lost: 417.806396
epoch 3,loss: 250.067520 test_lost: 239.161896
epoch 4,loss: 249.070770 test_lost: 240.172562
epoch 5,loss: 249.041687 test_lost: 241.138657
epoch 6,loss: 248.962387 test_lost: 240.121490
epoch 7,loss: 249.519333 test_lost: 280.088928
epoch 8,loss: 249.516296 test_lost: 276.279449
epoch 9,loss: 248.916580 test_lost: 245.475571
epoch 10,loss: 249.164062 test_lost: 262.602539
epoch 11,loss: 248.728790 test_lost: 239.911713
epoch 12,loss: 249.856491 test_lost: 305.030579
epoch 13,loss: 248.634506 test_lost: 239.246948
epoch 14,loss: 248.723221 test_lost: 247.103622
epoch 15,loss: 248.655701 test_lost: 239.415237
epoch 16,loss: 248.529251 test_lost: 240.258789
epoch 17,loss: 248.554565 test_lost: 238.131119
epoch 18,loss: 248.464111 test_lost: 240.331085
epoch 19,loss: 248.491501 test_lost: 248.449249
epoch 20,loss: 248.773087 test_lost: 269.565887
epoch 1,loss: 356345.781250 test_lost: 1149797

In [166]:
dense = net[0]
dense.weight.data()


[[ 2.4541709e-01  2.2653717e-01  2.6043013e-01  2.7122605e-01
  -1.5583912e-04]]
<NDArray 1x5 @cpu(0)>


[[0.24838287 0.25007543 0.26047033 0.23182087 0.00031211]]
<NDArray 1x5 @cpu(0)>

In [167]:
dense.bias.data()


[0.00020585]
<NDArray 1 @cpu(0)>


[0.00020866]
<NDArray 1 @cpu(0)>